In [9]:
# Making imports convenient
import sys
import os
PATH=os.getcwd().split('/notebooks')[0]
sys.path.insert(1, PATH)

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset,concatenate_datasets
import transformers
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

from transformers import AutoTokenizer, DataCollatorWithPadding,AutoModelForSequenceClassification,AdamW,get_scheduler,TrainingArguments,Trainer

from src.utils.myutils import *
import yaml

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

CS_DATA_PATH = PATH + '/data/CS/processed/BABE/train.csv'
CONFIG_PATH = PATH + '/src/utils/config.yaml'

BATCH_SIZE = 64
transformers.utils.logging.set_verbosity_error()

### BABE train_test split (SKIP IF DONE)

In [ ]:
babe = load_dataset("csv", data_files=PATH + '/data/CS/raw/BABE/SG2.csv')['train']

In [ ]:
babe = babe.train_test_split(0.15,seed=42)

In [54]:
babe['train'].to_csv(PATH + '/data/CS/processed/BABE/train.csv',index=False)
babe['test'].to_csv(PATH + '/data/CS/processed/BABE/test.csv',index=False) #THIS IS FOR THE FINAL MODEL SELECTED,TUNED

125614

## Load data

In [2]:
data = load_dataset('csv',data_files = CS_DATA_PATH)['train']
data

Using custom data configuration default-84520668e41f8432
Reusing dataset csv (/home/horyctom/.cache/huggingface/datasets/csv/default-84520668e41f8432/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


Dataset({
    features: ['sentence', 'label'],
    num_rows: 3122
})

In [3]:
with open(CONFIG_PATH) as f:
    config_data = yaml.load(f, Loader=yaml.FullLoader)

## Training

In [10]:
def compute_metrics_eval(eval_preds):
    metric = load_metric("f1")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [11]:
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [12]:
training_args = TrainingArguments(
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,  
    per_device_eval_batch_size=BATCH_SIZE,
    logging_steps=10,
    disable_tqdm = False,
    warmup_steps=10,
    save_total_limit=2,
    evaluation_strategy="steps",
    metric_for_best_model = 'f1',
    weight_decay=0.1,
    output_dir = './',
    learning_rate=5e-5)

## BEST PARAMS

### Cross-Val all models

In [13]:
model_scores = {}

for model_name in config_data['models']:
    scores = []
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False,padding=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    token_full = preprocess_data(data,tokenizer,'sentence')

    print("Running 5-fold CV on model: ",model_name,"...")
    for train_index, val_index in skfold.split(token_full['input_ids'],token_full['label']):

        token_train = Dataset.from_dict(token_full[train_index])
        token_valid = Dataset.from_dict(token_full[val_index])


        model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
        model.to(device)
        trainer = Trainer(model,training_args,train_dataset=token_train,eval_dataset=token_valid,compute_metrics=compute_metrics_eval,data_collator=data_collator,
                          tokenizer=tokenizer)
        trainer.train()

        #evaluation
        eval_dataloader = DataLoader(token_valid, batch_size=BATCH_SIZE, collate_fn=data_collator)
        scores.append(compute_metrics(model,device,eval_dataloader)['f1'])
        
    print("Done.")
    model_scores[model_name] = scores


Loading cached processed dataset at /home/horyctom/.cache/huggingface/datasets/csv/default-84520668e41f8432/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-f89f73809237ea4b.arrow
/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, co

Running 5-fold CV on model:  UWB-AIR/Czert-B-base-cased ...


***** Running training *****
  Num examples = 2497
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 120


Step,Training Loss,Validation Loss,F1
10,0.669600,0.617040,0.697872
20,0.650400,0.621123,0.733333
30,0.580400,0.535330,0.730832
40,0.725600,0.545324,0.750365
50,0.506600,0.543201,0.681481
60,0.460200,0.537064,0.711191
70,0.434000,0.517357,0.752475
80,0.485100,0.503969,0.720280
90,0.368200,0.536824,0.736301
100,0.312900,0.588545,0.739353


***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64


Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/format

...

Running 5-fold CV on model:  xlm-roberta-large ...


loading configuration file https://huggingface.co/xlm-roberta-large/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/4d7a1550c9ab8701667bc307a1213c040fcc06dc87a5e4994e72aecc0d7e0337.842c7737719967568f4691849854475018d6cf7ce21f52576bb6e0d10091bd3c
Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file https://huggingface

Step,Training Loss,Validation Loss,F1
10,0.740500,0.708324,0.657358


***** Running Evaluation *****
  Num examples = 625
  Batch size = 64


RuntimeError: CUDA out of memory. Tried to allocate 150.00 MiB (GPU 0; 31.75 GiB total capacity; 29.67 GiB already allocated; 84.00 MiB free; 30.82 GiB reserved in total by PyTorch)

In [14]:
for model in confif_data['models']:
    model_scores

{'UWB-AIR/Czert-B-base-cased': [0.7568,
  0.752,
  0.7275641025641025,
  0.7371794871794872,
  0.7451923076923076],
 'ufal/robeczech-base': [0.7824,
  0.7664,
  0.7740384615384616,
  0.7884615384615384,
  0.7788461538461539],
 'bert-base-multilingual-cased': [0.7776,
  0.7296,
  0.7099358974358975,
  0.7612179487179487,
  0.7451923076923076],
 'fav-kky/FERNET-C5': [0.7856,
  0.7984,
  0.7548076923076922,
  0.7948717948717948,
  0.7580128205128205],
 'fav-kky/FERNET-News': [0.5392,
  0.688,
  0.5096153846153846,
  0.7884615384615384,
  0.5641025641025641],
 'DeepPavlov/bert-base-bg-cs-pl-ru-cased': [0.7648,
  0.7456,
  0.7692307692307693,
  0.7740384615384616,
  0.7612179487179487]}